### ML CHALLENGE 3 HACKEREARTH
#### Predict add Clicks



In [15]:
#load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
#import datasets
training_df = pd.read_csv('D:/python/ML-challenge3/train.csv')
test_df = pd.read_csv('D:/python/ML-challenge3/test.csv')

In [29]:
training_df.head()

,ID,datetime,siteid,offerid,category,merchant,countrycode,browserid,devid,click,tweekday,thour
0,IDsrk7SoW,2017-01-14 09:42:09,4709696.0,887235,17714,20301556,e,Firefox,-999,0,5,9
1,IDmMSxHur,2017-01-18 17:50:53,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0,2,17
2,IDVLNN0Ut,2017-01-11 12:46:49,98480.0,518539,25085,2050923,a,Edge,-999,0,2,12
3,ID32T6wwQ,2017-01-17 10:18:43,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0,1,10
4,IDqUShzMg,2017-01-14 16:02:33,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0,5,16


In [10]:
training_df.shape

(12137810, 10)

In [8]:
training_df.isnull().sum()

ID                   0
datetime             0
siteid         1212515
offerid              0
category             0
merchant             0
countrycode          0
browserid       608327
devid          1820299
click                0
dtype: int64

In [13]:
#replace NA values with -999
training_df.siteid.fillna(-999,inplace=True)
training_df.browserid.fillna(-999,inplace=True)
training_df.devid.fillna(-999,inplace=True)

test_df.siteid.fillna(-999,inplace=True)
test_df.browserid.fillna(-999,inplace=True)
test_df.devid.fillna(-999,inplace=True)

In [14]:
training_df.isnull().sum()

ID             0
datetime       0
siteid         0
offerid        0
category       0
merchant       0
countrycode    0
browserid      0
devid          0
click          0
dtype: int64

In [19]:
#datetime handeling

#set datetime
training_df['datetime'] = pd.to_datetime(training_df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])


In [37]:
#training_df['tweekday'] = training_df['datetime'].dt.weekday
training_df['thour'] = training_df['datetime'].dt.hour

#test_df['tweekday'] = test_df['datetime'].dt.weekday
test_df['thour'] = test_df['datetime'].dt.hour

In [32]:
#replace days 1,2,3,4 with weekday, and 5,6,7 with weekend
weekdays = [0,1,2,3]
weekend = [4,5,6]

training_df.loc[training_df.tweekday.isin(weekdays), 'tweekday'] = 'weekday'
training_df.loc[training_df.tweekday.isin(weekend),'tweekday'] = 'weekend'

In [34]:
test_df.loc[test_df.tweekeday.isin(weekdays), 'tweekday'] = 'weekday'
test_df.loc[test_df.tweekeday.isin(weekdays),'tweekday'] = 'weekday'

In [42]:
#same thing with hours
day = [8,9,10,11,12]
afternoon=[13,14,15,16,17,18]
evening = [19,20,21,22,23,24]
night=[1,2,3,4,5,6,7]

training_df.loc[training_df.thour.isin(day),'thour'] = 'day'
training_df.loc[training_df.thour.isin(afternoon),'thour'] ='afternoon'
training_df.loc[training_df.thour.isin(evening),'thour'] ='evening'
training_df.loc[training_df.thour.isin(night),'thour'] ='night'

In [39]:
test_df.loc[test_df.thour.isin(day),'thour'] = 'day'
test_df.loc[test_df.thour.isin(afternoon),'thour'] = 'afternoon'
test_df.loc[test_df.thour.isin(evening),'thour'] = 'evening'
test_df.loc[test_df.thour.isin(night),'thour'] = 'night'

In [49]:
#training_df.drop('datetime', axis=1, inplace=True)
training_df.dtypes
cols = ['offerid', 'category', 'merchant', 'siteid']

for i in cols:
    training_df[i] = training_df[i].astype('object')
    test_df[i] = test_df[i].astype('object')
    

In [77]:
categoricals = cols + ['countrycode', 'devid', 'browserid', 'thour', 'tweekday']
#replace values with labelencoder
from sklearn.preprocessing import LabelEncoder

for x in categoricals:
    le = LabelEncoder()
    le.fit(list(training_df[x].values) + list(test_df[x].values))
    training_df[x] = le.transform(list(training_df[x].values))
    test_df[x] = le.transform(list(test_df[x].values))

In [61]:
training_df.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,tweekday,thour
0,128865,784773,48,127,4,3,0,0,weekend,day
1,142053,157563,59,65,1,9,1,0,weekday,afternoon
2,2618,458279,69,15,0,2,0,0,weekday,day
3,243406,345067,117,507,2,3,2,0,weekday,day
4,154278,417948,36,276,3,9,1,0,weekend,afternoon


### Model training

In [78]:
# remove cols that we don't need 
#training_df.drop('ID', axis=1, inplace=True)
#test_df.drop('ID', axis=1, inplace=True)

from  sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(training_df.loc[:,training_df.columns != 'click'], training_df['click'], test_size = 0.5)

In [72]:
import lightgbm as lgb

In [79]:
X_train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,tweekday,thour
344246,49403,756024,117,255,5,4,2,1,1
3840512,235511,839615,34,522,1,8,1,2,1
8500062,0,624953,130,633,4,1,1,1,4
8504405,183731,391954,231,477,2,2,3,2,1
8712057,256489,670418,238,351,4,3,2,2,1


In [92]:
#test_df.drop('datetime',inplace = True, axis=1)
#test_df.drop('tweekday',inplace = True, axis=1)
test_df.columns=['siteid','offerid','category','merchant','coutrycode','browserid','devid','tweekday','thour']

In [82]:
dtrain = lgb.Dataset(X_train, Y_train)
dval = lgb.Dataset(X_test, Y_test)

params = {
    
    'num_leaves' : 256,
    'learning_rate':0.03,
    'metric':'auc',
    'objective':'binary',
    'early_stopping_round': 40,
    'max_depth':10,
    'bagging_fraction':0.5,
    'feature_fraction':0.6,
    'bagging_seed':2017,
    'feature_fraction_seed':2017,
    'verbose' : 1
    
    
}

clf = lgb.train(params, dtrain,num_boost_round=500,valid_sets=dval,verbose_eval=20)

[20]	valid_0's auc: 0.96929
[40]	valid_0's auc: 0.969692
[60]	valid_0's auc: 0.969854
[80]	valid_0's auc: 0.96997
[100]	valid_0's auc: 0.969907
[120]	valid_0's auc: 0.970331
[140]	valid_0's auc: 0.970593
[160]	valid_0's auc: 0.970793
[180]	valid_0's auc: 0.971068
[200]	valid_0's auc: 0.971378
[220]	valid_0's auc: 0.971565
[240]	valid_0's auc: 0.971713
[260]	valid_0's auc: 0.971902
[280]	valid_0's auc: 0.972116
[300]	valid_0's auc: 0.9722
[320]	valid_0's auc: 0.972305
[340]	valid_0's auc: 0.972385
[360]	valid_0's auc: 0.972487
[380]	valid_0's auc: 0.97264
[400]	valid_0's auc: 0.972783
[420]	valid_0's auc: 0.972886
[440]	valid_0's auc: 0.97303
[460]	valid_0's auc: 0.973259
[480]	valid_0's auc: 0.973441
[500]	valid_0's auc: 0.973613


In [93]:
preds = clf.predict(test_df.loc[:,test_df.columns != 'click'])

In [97]:
test_df_full = pd.read_csv('D:/python/ML-challenge3/test.csv')
sub = pd.DataFrame({'ID':test_df_full['ID'], 'click':preds})
sub.to_csv('lgb_pyst.csv', index=False)